In [12]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

In [13]:
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input
from keras.models import Model
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [22]:
imgs = np.load('75/images.npy')
imgs = imgs.astype('float32')
indices = np.random.permutation(imgs.shape[0])
imgs = imgs[indices]

split_1 = int(18000*0.8)
split_2 = int(18000*0.9)

train_imgs = imgs[:split_1]
val_imgs = imgs[split_1:split_2]
test_imgs = imgs[split_2:]

train_imgs = train_imgs / 255.0
test_imgs = test_imgs / 255.0
val_imgs = val_imgs / 255.0

labels = np.load('75/labels.npy')
labels = labels.astype('int32')
labels = labels[indices]
train_labels = labels[:split_1]
val_labels = labels[split_1:split_2]
test_labels = labels[split_2:]

train_imgs = train_imgs.reshape((train_imgs.shape[0], 75, 75, 1))
val_imgs = val_imgs.reshape((val_imgs.shape[0], 75, 75, 1))
test_imgs = test_imgs.reshape((test_imgs.shape[0], 75, 75, 1))

In [23]:
def conv_time(time):
    ntime = 0
    if time[1] > 30:
        ntime = (time[0] + 0.5)
    else:
        ntime = time[0]
    return ntime

train_labels_converted = np.array([conv_time(time) for time in train_labels])
test_labels_converted = np.array([conv_time(time) for time in test_labels])
val_labels_converted = np.array([conv_time(time) for time in val_labels])

encoder = LabelEncoder()
test_labels_encoded = encoder.fit_transform(test_labels_converted.reshape(-1))
train_labels_encoded = encoder.fit_transform(train_labels_converted.reshape(-1))
val_labels_encoded = encoder.fit_transform(val_labels_converted.reshape(-1))

OHencoder = OneHotEncoder(sparse_output=False)
train_labels_oh = OHencoder.fit_transform(train_labels_encoded.reshape(-1, 1))
val_labels_oh = OHencoder.fit_transform(val_labels_encoded.reshape(-1, 1))

In [24]:
# data augmentation (as images are turned/differ in brightness/position)
aug = datagen = ImageDataGenerator(
    rotation_range=30,       # Randomly rotate images by up to 30 degrees
    zoom_range=0.2,          # Randomly zoom in on images
    width_shift_range=0.1,   # Randomly shift images horizontally
    height_shift_range=0.1,  # Randomly shift images vertically
    brightness_range=[0.8, 1.2]     # Adjust brightness
)

datagen.fit(train_imgs)

In [28]:
# simplified model
input_shape = (75, 75, 1)
model = keras.models.Sequential()

# Convolutional layers with simplified architecture
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

# Dense layers
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

# Output layer (24 categories for 24 time intervals)
model.add(Dense(24, activation='softmax'))

In [29]:
# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.01) #higher learning rate than set before
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])

# Callback
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Summary
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 73, 73, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 34, 34, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 15, 15, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │     1,605,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 24)             │         1,560 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,741,272 (6.64 MB)

 Trainable params: 1,741,272 (6.64 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
# Train the model using the augmented data, 12 epochs for now
model.fit(datagen.flow(train_imgs, train_labels_oh, batch_size=256),
        epochs=12, validation_data=(val_imgs, val_labels_oh),
        callbacks=[early_stop])

Epoch 1/12
57/57 ━━━━━━━━━━━━━━━━━━━━ 30s 489ms/step - accuracy: 0.0393 - loss: 3.1804 - val_accuracy: 0.0406 - val_loss: 3.1791
Epoch 2/12
57/57 ━━━━━━━━━━━━━━━━━━━━ 27s 477ms/step - accuracy: 0.0453 - loss: 3.1781 - val_accuracy: 0.0444 - val_loss: 3.1791
Epoch 3/12
57/57 ━━━━━━━━━━━━━━━━━━━━ 29s 509ms/step - accuracy: 0.0375 - loss: 3.1775 - val_accuracy: 0.0400 - val_loss: 3.1791
Epoch 4/12
57/57 ━━━━━━━━━━━━━━━━━━━━ 28s 492ms/step - accuracy: 0.0406 - loss: 3.1777 - val_accuracy: 0.0383 - val_loss: 3.1794
Epoch 5/12
57/57 ━━━━━━━━━━━━━━━━━━━━ 30s 530ms/step - accuracy: 0.0440 - loss: 3.1782 - val_accuracy: 0.0411 - val_loss: 3.1789
Epoch 6/12
57/57 ━━━━━━━━━━━━━━━━━━━━ 29s 514ms/step - accuracy: 0.0408 - loss: 3.1782 - val_accuracy: 0.0394 - val_loss: 3.1791
Epoch 7/12
57/57 ━━━━━━━━━━━━━━━━━━━━ 30s 517ms/step - accuracy: 0.0477 - loss: 3.1771 - val_accuracy: 0.0444 - val_loss: 3.1787
Epoch 8/12
57/57 ━━━━━━━━━━━━━━━━━━━━ 30s 529ms/step - accuracy: 0.0424 - loss: 3.1776 - val_accu

In [32]:
loss, accuracy = model.evaluate(val_imgs, val_labels_oh) # I used evaluate intead of predict
print(f"Validation loss: {loss}, Validation accuracy: {accuracy}")

57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.0453 - loss: 3.1769
Validation loss: 3.1783668994903564, Validation accuracy: 0.04055555537343025


Not better than your model i think